In [20]:
import sys
import math

def next_line(file):
    line = file.readline()
    return line

# file containing distance and position stats
file = open("0ipdqtest.txt", "r")
data = next_line(file).strip('\n')

numVehs = 0

# file containing the vehicle IDs where each line is in the form x_y, where
# x = mobility->getExternalId() and the module name in the .anf file is
# ConfigCollectionnetwork.node[y].appl
ids = open("0idsqtest.txt", "r")
vDict = {} # this will contain the link between x and y in each line in the ID file

idLine = next_line(ids).strip('\n')
while idLine:
	u = idLine.find('_')
	vId = idLine[u+1:]
	fId = idLine[0:u]
	vDict[fId] = eval(vId)
	idLine = next_line(ids).strip('\n')
	numVehs += 1
	
ipdList = [] # contains IPD data for each sender-receiver vehicle pair
for i in range(0, numVehs):
	ipdList.append([-1 for j in range(0, numVehs)])
print("Now scanning input")

# this file can be used to write each IPD and distance value to; can be commented out if not needed
#output = open("output4lanetenhz20mw200.txt", "w")

# distance ranges; these are optional and can be commented out
# to be used when you want the total number of BSMs received within 50m,
# 100m, 200m, 300m, and >300m by all vehicles
n0 = 0
n50 = 0
n100 = 0
n200 = 0
n300 = 0

# this list is to be used if you want to get the average IPD for consistent
# distance intervals; number of intervals can be changed as needed.
ipdIntervals = [[] for i in range(0,35)]

# individual vehicle IPD data; these are optional and can be commented out
# to be used when you want to get IPD data on specific vehicles
ipd50 = []
ipd100 = []
ipd150 = []
ipd200 = []
ipd539 = []

maxDist = 0
maxGap = 0
last = -1

j = 1
while data:
	if data[:10] == 'undisposed':
		data = next_line(file).strip('\n')
		continue
	
	# divides each line in the distance file into the individual data pieces
	first = data.find('_')
	second = data.find('_', first+1)
	third = data.find('_', second+1)
	fourth = data.find('_', third+1)
	fifth = data.find('_', fourth+1)
	sixth = data.find('_', fifth+1)
	seventh = data.find('_', sixth+1)
	senderX = eval(data[:first])
	senderY = eval(data[first+1:second])
	senderID = data[second+1:third]
	receiverX = eval(data[third+1:fourth])
	receiverY = eval(data[fourth+1:fifth])
	receiverID = data[fifth+1:sixth]
	simTime = eval(data[seventh+1:])
	distance = round(math.sqrt((senderX-receiverX)**2 + (senderY-receiverY)**2), 5) # calculates distance between sending and receiving vehicle
	
	"""if last == -1:
		last = simTime
	else:
		if simTime - last > maxGap:
			maxGap = simTime - last
		last = simTime"""

# this block is only to be used with the distance range variables; otherwise it can be commented out
# the distance values can be changed as needed
	if distance <= 50:
		n0 += 1
	elif distance <= 100:
		n50 += 1
	elif distance <= 200:
		n100 += 1
	elif distance <= 300:
		n200 += 1
	else:
		n300 += 1
	
	lastTime = ipdList[vDict[receiverID]][vDict[senderID]]
	if lastTime != -1:
		ipd = simTime - lastTime
	else:
		ipd = -1
		
	if distance > maxDist:
		maxDist = distance
		
# this section is only to be used with the individual vehicle IPD data; otherwise it can be commented out
# vehicle names and if condition can be changed as needed
		"""if senderID == 'sender' and 'receiver' in receiverID:
			if receiverID == 'receiver50':
				ipd50.append(ipd)
			elif receiverID == 'receiver100':
				ipd100.append(ipd)
			elif receiverID == 'receiver150':
				ipd150.append(ipd)
			elif receiverID == 'receiver200':
				ipd200.append(ipd)
			elif receiverID == 'receiver539':
				ipd539.append(ipd)"""
			
# this section is only used if you are writing the IPD and distance values
# to the output file; otherwise it can be commented out
		"""output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + str(ipd) + '\n')
		output.flush()"""
		
# this section is used only if you want to get the average IPD over consistent
# distance intervals; otherwise it can be commented out
# the number of intervals and length of each interval can be changed as needed
	for i in range(1, len(ipdIntervals)+1):
		if 20*i > distance and ipd != -1:
			ipdIntervals[i-1].append(ipd)
			break

# this section is only used if you are writing the IPD and distance values
# to the output file and only if you want to include the first time a
# sender-receiver pair exchanges BSMs; otherwise it can be commented out		
	"""else:
		output.write(senderID + '\t' + receiverID + '\t' + str(distance) + '\t' + "First" + '\n')"""
	
	
	ipdList[vDict[receiverID]][vDict[senderID]] = simTime
	data = next_line(file).strip('\n')
	
"""
	# this block tells you how many lines the program has gone through so you can
	# see how much progress the program is making
	j+=1
	if j%100000 == 0:
		print(j)
"""

# print IPD data for individual vehicles if used
"""print(sum(ipd50)/len(ipd50), len(ipd50))
print(sum(ipd100)/len(ipd100), len(ipd100))
print(sum(ipd150)/len(ipd150), len(ipd150))
print(sum(ipd200)/len(ipd200), len(ipd200))
print(sum(ipd539)/len(ipd539), len(ipd539))"""

# print number of BSMs received in distance ranges if used
print(n0, n50, n100, n200, n300)

# print average IPD for each distance interval if ipdIntervals is used
print("**********")
for i in range(0, len(ipdIntervals)):
	if len(ipdIntervals[i]) > 0:
		ipdIntervals[i] = sum(ipdIntervals[i])/len(ipdIntervals[i])
	else:
		ipdIntervals[i] = 'N/A'
	print(str(ipdIntervals[i]))

#print(maxGap)
#print(maxDist)

file.close()
#output.close()
#ids.close()



Now scanning input
106490 158187 219213 2 0
**********
0.4007820944521055
0.40079904995485555
0.40080184685587733
0.40167893838678637
0.401885189412219
0.4037733226587717
0.41344901499952386
0.4285556364319702
0.5676372346928661
1.498982576804922
0.8000032669920021
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
N/A
